# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
# import os
# import pytranslate

import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [2]:
# # Set the default encoding to UTF-8
# os.environ['HTTP_ENCODING'] = 'utf-8'

# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,shahr-e babak,30.1165,55.1186,83.61,15,41,14.27,IR,23/07/22
1,1,mar del plata,-38.0023,-57.5575,54.09,91,100,11.50,AR,23/07/22
2,2,talnakh,69.4865,88.3972,62.24,96,100,5.84,RU,23/07/22
3,3,copala,16.6500,-98.9833,88.29,60,45,7.78,MX,23/07/22
4,4,adamstown,-25.0660,-130.1015,69.44,67,21,17.87,PN,23/07/22


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    # hover_cols = ["", "Max Temp"]
)

# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
vaca_data_df = city_data_df[(city_data_df['Max Temp'] > 55) & (city_data_df['Max Temp'] < 95) & (city_data_df['Cloudiness'] < 15) & (city_data_df['Humidity'] < 25)]


# Drop any rows with null values
vaca_data_df.dropna(inplace=True)

# Display sample data
vaca_data_df.reset_index(drop=True, inplace=True)
vaca_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,133,hun,29.1268,15.9477,91.56,20,0,16.60,LY,23/07/22
1,150,tamanrasset,22.7850,5.5228,85.91,10,0,4.61,DZ,23/07/22
2,155,oltinko'l,40.8011,72.1634,83.16,18,0,4.25,UZ,23/07/22
3,278,karratha,-20.7377,116.8463,62.11,23,0,20.49,AU,23/07/22
4,280,yauya,-8.9833,-77.3000,64.72,23,0,3.56,PE,23/07/22
5,313,baharly,38.4362,57.4316,86.00,18,0,0.40,TM,23/07/22
6,346,murzuq,25.9155,13.9184,91.85,15,0,10.25,LY,23/07/22
7,389,waddan,29.1614,16.1390,90.79,22,0,19.08,LY,23/07/22
8,400,ubari,26.5921,12.7805,92.43,13,0,9.78,LY,23/07/22
9,427,vernon,50.2581,-119.2691,94.75,16,5,6.24,CA,23/07/22


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vaca_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.reset_index(drop=True, inplace=True)
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,hun,LY,29.1268,15.9477,20,
1,tamanrasset,DZ,22.7850,5.5228,10,
2,oltinko'l,UZ,40.8011,72.1634,18,
3,karratha,AU,-20.7377,116.8463,23,
4,yauya,PE,-8.9833,-77.3000,23,
5,baharly,TM,38.4362,57.4316,18,
6,murzuq,LY,25.9155,13.9184,15,
7,waddan,LY,29.1614,16.1390,22,
8,ubari,LY,26.5921,12.7805,13,
9,vernon,CA,50.2581,-119.2691,16,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
param = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    # "format": "json",
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    param["filter"] = f"circle:{longitude},{latitude},{radius}"
    param["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=param)
    
    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    
#     # Translate the hotel names to English
#     translator = pytranslate.Translator()
#     hotel_names_english = []
#     for hotel_name in hotel_names:
#         translated_hotel_name = translator.translate(hotel_name, dest='en')
#         hotel_names_english.append(translated_hotel_name)
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.reset_index(drop=True, inplace=True)
hotel_df

Starting hotel search
hun - nearest hotel: فندق الهروج
tamanrasset - nearest hotel: فندق الأمان
oltinko'l - nearest hotel: No hotel found
karratha - nearest hotel: Karratha International Hotel
yauya - nearest hotel: No hotel found
baharly - nearest hotel: No hotel found
murzuq - nearest hotel: فندق باريس - مرزق
waddan - nearest hotel: No hotel found
ubari - nearest hotel: فندق اوباري
vernon - nearest hotel: Okanagan Royal Park Inn
illizi - nearest hotel: بوناقة
ghat - nearest hotel: فندق تاسيلي
robore - nearest hotel: Lajas
navoiy - nearest hotel: Biryuza
chapada dos guimaraes - nearest hotel: Pousada Rios
diamantino - nearest hotel: Hotel kayaby


,City,Country,Lat,Lng,Humidity,Hotel Name
0,hun,LY,29.1268,15.9477,20,فندق الهروج
1,tamanrasset,DZ,22.7850,5.5228,10,فندق الأمان
2,oltinko'l,UZ,40.8011,72.1634,18,No hotel found
3,karratha,AU,-20.7377,116.8463,23,Karratha International Hotel
4,yauya,PE,-8.9833,-77.3000,23,No hotel found
5,baharly,TM,38.4362,57.4316,18,No hotel found
6,murzuq,LY,25.9155,13.9184,15,فندق باريس - مرزق
7,waddan,LY,29.1614,16.1390,22,No hotel found
8,ubari,LY,26.5921,12.7805,13,فندق اوباري
9,vernon,CA,50.2581,-119.2691,16,Okanagan Royal Park Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)